<a href="https://colab.research.google.com/github/adalves-ufabc/2023.Q1-PLN/blob/main/2023_Q1_PLN_Notebook_House_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2023.Q1]**
Prof. Alexandre Donizeti Alves

## **PROJETO PRÁTICO [Série House]**
---



**Conjunto de Dados**

In [ ]:
import requests
import zipfile
from io import BytesIO

# ID do arquivo zip no Google Drive
file_id = "1_rAzomhikVwtK32LV_yGhTKGuUsoEQ10"

# URL de download do arquivo zip
url = f"https://drive.google.com/uc?id={file_id}&export=download"

# Faz o download do arquivo zip
response = requests.get(url)
file_bytes = BytesIO(response.content)

# Extrai o arquivo zip
with zipfile.ZipFile(file_bytes, "r") as zip_ref:
    zip_ref.extractall()

In [ ]:
import pandas as pd

# Lê o arquivo Excel em um DataFrame
df = pd.read_excel('/content/house_season_1-8.xlsx')

df

,season,num_episode,episode,title,director,character,transcript
0,1,1,1.01,Pilot,Bryan Singer,Melanie,Why are you late?
1,1,1,1.01,Pilot,Bryan Singer,Rebecca,You’re not going to like the answer.
2,1,1,1.01,Pilot,Bryan Singer,Melanie,I already know the answer.
3,1,1,1.01,Pilot,Bryan Singer,Rebecca,I missed the bus.
4,1,1,1.01,Pilot,Bryan Singer,Melanie,"I don’t doubt it, no bus stops near Brad’s. Yo..."
...,...,...,...,...,...,...,...
76460,8,177,8.22,Everybody Dies,David Shore,House,Just switched the dental records.
76461,8,177,8.22,Everybody Dies,David Shore,Wilson,You're destroying your entire life. You can't ...
76462,8,177,8.22,Everybody Dies,David Shore,House,"I'm dead, Wilson. How do you want to spend you..."
76463,8,177,8.22,Everybody Dies,David Shore,Wilson,When the cancer starts getting really bad…


**POS Tagging**

In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import spacy

def word_freq(df, seasons, tags, n=None):
    # Filtra as temporadas desejadas
    df_seasons = df[df['season'].isin(seasons)]
    
    # Junta os textos de cada episódio da temporada selecionada
    episodes_text = df_seasons.groupby(['season', 'episode'])['transcript'].apply(lambda x: ' '.join(x)).reset_index()

    # Carrega o modelo em inglês do spaCy
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    
    # Função para processar o texto e extrair as tags
    def extract_nouns(text):
        doc = nlp(text)
        return [token.text for token in doc if token.pos_ in tags]

    # Aplica a função para extrair as tags de cada episódio
    episodes_text['tags'] = episodes_text['transcript'].apply(extract_nouns)
    
    # Une as listas de palavras de todos os episódios em uma única lista
    words = [word for episode_words in episodes_text['tags'].tolist() for word in episode_words]

    # stopwords em inglês
    stop_words = stopwords.words('english')

    lemmatizer = WordNetLemmatizer()

    # Remove as stopwords e lematiza
    words_filtered = [lemmatizer.lemmatize(token) for token in words if token.isalpha() and token.lower() not in stop_words]
    
    # Conta as palavras e pega as N mais frequentes
    word_freq = dict(Counter(words_filtered).most_common(n))

    # Ordena as palavras em ordem decrescente de frequência
    word_freq = dict(sorted(word_freq.items(), key=lambda x: x[1], reverse=True))
    
    return word_freq

In [ ]:
# n palavras mais frequentes em determinadas temporadas de acordo com o(s) tipo(s) de tag(s)
word_freq(df, [1,2], ['NOUN'], 10)

{'time': 454,
 'patient': 426,
 'thing': 420,
 'blood': 375,
 'guy': 371,
 'way': 337,
 'test': 317,
 'year': 309,
 'doctor': 296,
 'people': 296}

In [ ]:
import pandas as pd

def find_trasncripts(df, search_terms, seasons, n=None):
    # Filter the DataFrame by season and search terms
    results = df[df['season'].isin(seasons) & df['transcript'].str.contains('|'.join(search_terms), case=False)]
    
    # Sort the results by season and episode
    results = results.sort_values(['season', 'episode'])
    
    # Return the top n lines of dialogue
    return results['transcript'][:n]

In [ ]:
find_trasncripts(df, ['what cause'], [1,2,3,4,5,6,7,8], 10)

5557     Purulent sputum, dyspnea, bronchi bilaterally....
5720                         We don’t know what causes it.
7161     At this point it doesn’t matter what caused th...
7356                          W-what caused my s-symptoms?
9291                        And brown.  What causes brown?
11705    We think it may have been what caused her car ...
11750    Thank you for taking no interest in my mother....
11857    So, all we have to answer is what causes a 22-...
14328    It's not the surgery, it's the secrecy! What c...
15210    If I didn't wake him, I wouldn't have learned ...
Name: transcript, dtype: object

In [ ]:
import spacy
import pandas as pd
from collections import Counter

nlp = spacy.load('en_core_web_sm')

def find_nouns(df, search_terms, tags, seasons=None, n=None):
    # Filtre o dataframe pelas temporadas especificadas, se fornecidas
    if seasons:
        df = df[df['season'].isin(seasons)]

    # Pesquise os termos de busca no dataframe e extraia as falas correspondentes
    search_results = df[df['transcript'].str.contains('|'.join(search_terms), case=False)]

    # Utilize a biblioteca spaCy para identificar os substantivos nas falas correspondentes
    nouns = []
    for text in search_results['transcript']:
        doc = nlp(text)
        nouns.extend([token.lemma_ for token in doc if token.pos_ in tags])
    
    # Use o Counter para contar a frequência dos substantivos encontrados
    freq = dict(Counter(nouns))
    
    # Ordene os termos por frequência e selecione apenas os n termos mais frequentes
    sorted_terms = sorted(freq.items(), key=lambda x: x[1], reverse=True)[:n]
    sorted_freq = {term: frequency for term, frequency in sorted_terms}
    
    return sorted_freq

In [ ]:
find_nouns(df, ['what cause'], ['NOUN'], [1,2,3,4,5,6,7,8], 10)

{'problem': 13,
 'heart': 12,
 'lung': 9,
 'failure': 9,
 'symptom': 8,
 'liver': 8,
 'kidney': 7,
 'pain': 7,
 'loss': 6,
 'question': 4}

In [ ]:
import spacy
import pandas as pd
from collections import Counter

nlp = spacy.load('en_core_web_sm')

def find_tags_in_titles(df, tags, seasons=None, n=None):
    # Filtra o dataframe pelas temporadas especificadas, se fornecidas
    if seasons:
        df = df[df['season'].isin(seasons)]

    # Seleciona apenas as colunas relevantes
    df_title = df[['title']]
    df_title = df_title.drop_duplicates()['title']

    # Utilize a biblioteca spaCy para identificar as palavras de acordo com os tipos de tags nos títulos
    words = []
    for text in df_title:
        doc = nlp(text)
        words.extend([token.lemma_ for token in doc if token.pos_ in tags])
    
    # Use o Counter para contar a frequência das palavras encontradas
    freq = dict(Counter(words))
    
    # Ordene os termos por frequência e selecione apenas os n termos mais frequentes
    sorted_terms = sorted(freq.items(), key=lambda x: x[1], reverse=True)[:n]
    sorted_freq = {term: frequency for term, frequency in sorted_terms}
    
    return sorted_freq

In [ ]:
find_tags_in_titles(df, ['NOUN', 'ADJ', 'PROPN'], [1,2,3,4,5,6,7,8], 10)

{'House': 6,
 'love': 2,
 'story': 2,
 'TB': 2,
 'skin': 2,
 'lie': 2,
 'Euphoria': 2,
 'Será': 2,
 'Guy': 2,
 'word': 2}

**Reconhecimento de Entidades Nomeadas**

A técnica de Reconhecimento de Entidades Nomeadas (NER) pode ser usada na série "House" para identificar nomes próprios de pessoas, lugares e organizações mencionados nas falas dos personagens. Essas entidades podem ser úteis para entender as relações entre personagens, locais onde ocorrem os casos médicos e as organizações envolvidas no tratamento dos pacientes.

Por exemplo, o nome do hospital em que a série se passa, Princeton-Plainsboro Teaching Hospital, pode ser identificado como uma entidade nomeada. Além disso, os nomes dos personagens, como Gregory House e seus colegas médicos, também podem ser identificados como entidades nomeadas.

A análise de entidades nomeadas pode ser usada para entender melhor as relações entre os personagens e a frequência com que determinados locais ou organizações são mencionados. Isso pode ser útil para identificar os personagens mais relevantes na série, a localização geográfica dos casos médicos e os hospitais ou organizações médicas que são frequentemente mencionados na série.


Existem várias maneiras de usar NER na série "House". Algumas ideias incluem:

   * Identificação de personagens principais e secundários: usando NER, você pode identificar o nome dos personagens principais e secundários da série, bem como suas relações com outros personagens.

   * Identificação de locais: a série se passa no Princeton-Plainsboro Teaching Hospital, mas há outros locais que também são mencionados. Usando NER, você pode identificar esses locais e descobrir como eles se relacionam com a trama.
   
   * Análise de frequência de entidades nomeadas: usando NER, você pode analisar a frequência com que os personagens, locais e organizações são mencionados na série. Isso pode ajudar a identificar personagens ou locais importantes na trama.

   * Identificação de terminologia médica: a série usa muita terminologia médica, e NER pode ajudar a identificar esses termos e seu significado na trama.

   * Análise de sentimentos: além da identificação de entidades nomeadas, o NER também pode ser usado para analisar os sentimentos associados a essas entidades. Isso pode ajudar a entender melhor como os personagens se sentem sobre outros personagens, locais ou organizações.

**`spaCy Transformers — roBERTa`**

In [ ]:
# Download the spacy transformer (roberta-base) english model
!python -m spacy download en_core_web_trf

2023-04-24 17:10:51.708422: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
import spacy
import spacy_transformers

nlp = spacy.load("en_core_web_trf")

In [ ]:
def extract_named_entities_from_character(df, character, episode, types_entities):
    # Filtra o DataFrame pelas falas do personagem especificado no episódio especificado
    filtered_df = df[(df['episode'] == episode) & (df['character'] == character)]
    
    # Extrai as falas do personagem
    transcripts = filtered_df['transcript'].tolist()
    
    # Processa as falas com o modelo de linguagem do spaCy
    docs = [nlp(transcript) for transcript in transcripts]
    
    # Itera pelas entidades nomeadas e retorna um dicionário com as entidades e seus tipos
    named_entities = {}
    for doc in docs:
      for entity in doc.ents:
        if entity.label_ in types_entities:
            if entity.text not in named_entities:
                named_entities[entity.text] = set([entity.label_])
            else:
                named_entities[entity.text].add(entity.label_)
    
    return named_entities

In [ ]:
extract_named_entities_from_character(df, 'House', 1.01, ['PERSON', 'ORG', 'GPE'])

{'HMO': {'ORG'},
 'Trenton': {'GPE'},
 'Jagger': {'PERSON'},
 'House': {'ORG', 'PERSON'},
 'Wilson': {'PERSON'},
 'General Hospital': {'ORG'},
 'the New England Journal of Medicine': {'ORG'},
 'Felker': {'PERSON'},
 'Neurologist': {'PERSON'},
 'Chase': {'PERSON'}}

In [ ]:
def extract_named_entities_from_character_with_type_and_freq(df, character, episode, types_entities):
    # Filtra o DataFrame pelas falas do personagem especificado no episódio especificado
    filtered_df = df[(df['episode'] == episode) & (df['character'] == character)]
    
    # Extrai as falas do personagem
    transcripts = filtered_df['transcript'].tolist()
    
    # Processa as falas com o modelo de linguagem do spaCy
    docs = [nlp(transcript) for transcript in transcripts]
    
    # Itera pelas entidades nomeadas e retorna um dicionário com as entidades, seus tipos e suas frequências de ocorrência
    named_entities = {}
    for doc in docs:
        for entity in doc.ents:
            if entity.label_ in types_entities:
                named_entity = entity.text
                named_entity_type = entity.label_
                if named_entity_type not in named_entities:
                    named_entities[named_entity_type] = {}
                if named_entity not in named_entities[named_entity_type]:
                    named_entities[named_entity_type][named_entity] = 1
                else:
                    named_entities[named_entity_type][named_entity] += 1
    
    return named_entities

In [ ]:
spacy.explain('GPE')

'Countries, cities, states'

In [ ]:
named_entities = extract_named_entities_from_character_with_type_and_freq(df, 'House', 1.01, ['PERSON', 'ORG', 'GPE'])
named_entities

{'ORG': {'HMO': 1,
  'House': 1,
  'General Hospital': 1,
  'the New England Journal of Medicine': 1},
 'GPE': {'Trenton': 1},
 'PERSON': {'Jagger': 1,
  'Wilson': 1,
  'House': 2,
  'Felker': 1,
  'Neurologist': 1,
  'Chase': 2}}

In [ ]:
named_entities['PERSON']['House']

2

In [ ]:
named_entities.keys()

dict_keys(['ORG', 'GPE', 'PERSON'])

In [ ]:
tipos_entidades = list(named_entities.keys())
tipos_entidades[0]

'ORG'

In [ ]:
named_entities['PERSON']

{'Jagger': 1,
 'Wilson': 1,
 'House': 2,
 'Felker': 1,
 'Neurologist': 1,
 'Chase': 2}

In [ ]:
textos_org = list(named_entities['PERSON'].keys())
textos_org

['Jagger', 'Wilson', 'House', 'Felker', 'Neurologist', 'Chase']

In [ ]:
valores_org = list(named_entities['PERSON'].values())
valores_org

[1, 1, 2, 1, 1, 2]

**`Transformers`**

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.7 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

nlp = pipeline("ner", model="Davlan/distilbert-base-multilingual-cased-ner-hrl")

In [ ]:
def extract_named_entities_from_character_transformers(df, character, episode):
    # Filtra o DataFrame pelas falas do personagem especificado no episódio especificado
    filtered_df = df[(df['episode'] == episode) & (df['character'] == character)]
    
    # Extrai as falas do personagem
    transcripts = filtered_df['transcript'].tolist()
    
    # Processa as falas com o modelo de linguagem do Transformers
    docs = nlp(transcripts)
    
    # Cria uma lista vazia para armazenar as entidades nomeadas completas
    named_entities = []

    # Itera pelas entidades nomeadas e adiciona informações relevantes a um dicionário
    for doc in docs:
        for entity in doc:
            if entity['entity'] != 'O':
                entity_dict = {
                    'entity': entity['entity'],
                    'score': entity['score'],
                    'index': entity['index'],
                    'word': entity['word'],
                    'start': entity['start'],
                    'end': entity['end']
                }
                named_entities.append(entity_dict)
    
    return named_entities

In [ ]:
entities = extract_named_entities_from_character_transformers(df, 'House', 1.01)

for entity in entities:
  print(entity)

{'entity': 'B-LOC', 'score': 0.9996612, 'index': 43, 'word': 'Trento', 'start': 159, 'end': 165}
{'entity': 'I-LOC', 'score': 0.9993304, 'index': 44, 'word': '##n', 'start': 165, 'end': 166}
{'entity': 'B-LOC', 'score': 0.8416196, 'index': 1, 'word': 'Nice', 'start': 0, 'end': 4}
{'entity': 'B-PER', 'score': 0.99985504, 'index': 6, 'word': 'Jagger', 'start': 27, 'end': 33}
{'entity': 'B-PER', 'score': 0.86701584, 'index': 12, 'word': 'Duck', 'start': 25, 'end': 29}
{'entity': 'I-PER', 'score': 0.7007596, 'index': 13, 'word': '##lings', 'start': 29, 'end': 34}
{'entity': 'B-PER', 'score': 0.9983901, 'index': 16, 'word': 'House', 'start': 42, 'end': 47}
{'entity': 'B-PER', 'score': 0.9992774, 'index': 19, 'word': 'Wilson', 'start': 57, 'end': 63}
{'entity': 'B-PER', 'score': 0.8902711, 'index': 7, 'word': 'House', 'start': 13, 'end': 18}
{'entity': 'B-LOC', 'score': 0.9982241, 'index': 28, 'word': 'General', 'start': 91, 'end': 98}
{'entity': 'I-LOC', 'score': 0.99948525, 'index': 29, 'w

In [ ]:
def get_complete_entities(entities):
    # lista de entidades nomeadas completas
    complete_entities = []

    # entidade atualmente em construção
    current_entity = None

    # iterar sobre as entidades previstas
    for entity in entities:
        # se a entidade atual é nula ou a nova entidade começa com "B-"
        if current_entity is None or entity['entity'][0] == 'B':
            # armazena a entidade anterior se existir
            if current_entity is not None:
                if "#" in current_entity['word']:
                    current_entity['word'] = current_entity['word'].replace("#", "")
                complete_entities.append(current_entity)
            # começa a construir uma nova entidade
            current_entity = {
                'entity': entity['entity'][2:],
                'score': entity['score'],
                'start': entity['start'],
                'end': entity['end'],
                'word': entity['word'].replace("#", "")
            }
        # se a nova entidade começa com "I-" e a entidade atual existe
        elif current_entity is not None and entity['entity'][0] == 'I':
            # adiciona a palavra à entidade atual
            if "#" in entity['word']:
                current_entity['word'] += entity['word'].replace("#", "")
            else:
                current_entity['word'] += ' ' + entity['word']

            # atualiza o valor de "end" da nova entidade
            current_entity['end'] = entity['end']                

    # armazena a última entidade atual se existir
    if current_entity is not None:
        if "#" in current_entity['word']:
            current_entity['word'] = current_entity['word'].replace("#", "")
        complete_entities.append(current_entity)

    # retorna as entidades completas
    return complete_entities

In [ ]:
complete_entities = get_complete_entities(entities)

for entity in complete_entities:
  print(entity)

{'entity': 'LOC', 'score': 0.9996612, 'start': 159, 'end': 166, 'word': 'Trenton'}
{'entity': 'LOC', 'score': 0.8416196, 'start': 0, 'end': 4, 'word': 'Nice'}
{'entity': 'PER', 'score': 0.99985504, 'start': 27, 'end': 33, 'word': 'Jagger'}
{'entity': 'PER', 'score': 0.86701584, 'start': 25, 'end': 34, 'word': 'Ducklings'}
{'entity': 'PER', 'score': 0.9983901, 'start': 42, 'end': 47, 'word': 'House'}
{'entity': 'PER', 'score': 0.9992774, 'start': 57, 'end': 63, 'word': 'Wilson'}
{'entity': 'PER', 'score': 0.8902711, 'start': 13, 'end': 18, 'word': 'House'}
{'entity': 'LOC', 'score': 0.9982241, 'start': 91, 'end': 107, 'word': 'General Hospital'}
{'entity': 'ORG', 'score': 0.99960166, 'start': 48, 'end': 79, 'word': 'New England Journal of Medicine'}
{'entity': 'ORG', 'score': 0.9997682, 'start': 30, 'end': 38, 'word': 'Meth Lab'}
{'entity': 'PER', 'score': 0.98383015, 'start': 32, 'end': 35, 'word': 'Fel'}
{'entity': 'PER', 'score': 0.563545, 'start': 35, 'end': 38, 'word': 'ker'}
{'ent